### Getting a bit more deeper into Agents
#### Conversational ReAct Agents
This agent is designed for us in conversational settings. It incorporates the React framework to determine which tool to use and utilizes memory to remember pervious conversation interactions.

In [13]:
from langchain.agents import create_react_agent, load_tools, initialize_agent
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
import os

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.5, 
    google_api_key=os.getenv("GOOGLE_API_KEY"))

In [14]:
tools = load_tools(["llm-math"], llm=llm)

In [15]:
memory = ConversationBufferMemory(memory_key="chat_history")

conversational_agent = initialize_agent(
    agent="conversational-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    memory=memory,
    max_iterations=3)

In [16]:
output_1=conversational_agent.run("Add 7 to 9 and tell me the result.")
output_2=conversational_agent.run("Add 5 to the previous result")



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 7+9
Observation: Answer: 16
Thought:Do I need to use a tool? No
AI: The result of adding 7 to 9 is 16.
```

> Finished chain.


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 16+5
Observation: Answer: 21
Thought:Do I need to use a tool? No
AI: The result of adding 5 to the previous result (16) is 21.

> Finished chain.


#### ReAct Docstore

This agent utilizes the React framwork to communicate with a docstore. It requires the availability of a Search tool and a Lookup tool with identical names. The search tool is used to search for a document, while the Lookup tool looks up a term within the most recently found document.

A docstore in Langchain is essentially a storage system for documents. It provides a way for Langchain agents to access and interact with information. Like a Library

In [17]:
from langchain.agents import initialize_agent, Tool
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer

In [21]:
docstore = DocstoreExplorer(Wikipedia())
tools=[
    Tool(name="Search", func=docstore.search, description="A search engine that can be used to find information on Wikipedia."),
    Tool(name="Lookup", func=docstore.lookup, description="A search engine that can be used to find information on Wikipedia.")
]

# Initialize the agent 
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash", 
    temperature=0.5, 
    google_api_key=os.getenv("GOOGLE_API_KEY"))
    
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=8,
    handle_parsing_errors=True)

In [22]:
docstore_agent("Tell me a few key things about Mahatma Gandhi.")

/var/folders/y_/0dh_vnhx4h9bd_bn8cgqtnd00000gn/T/ipykernel_38212/836576075.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docstore_agent("Tell me a few key things about Mahatma Gandhi.")




> Entering new AgentExecutor chain...
Thought: The question asks for a few key things about Mahatma Gandhi. I need to search Mahatma Gandhi and find a few key things about him.
Action: Search[Mahatma Gandhi]
Observation: Mohandas Karamchand Gandhi (2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist who employed nonviolent resistance to lead the successful campaign for India's independence from British rule. He inspired movements for civil rights and freedom across the world. The honorific Mahātmā (from Sanskrit, meaning great-souled, or venerable), first applied to him in South Africa in 1914, is now used throughout the world.
Born and raised in a Hindu family in coastal Gujarat, Gandhi trained in the law at the Inner Temple in London and was called to the bar at the age of 22. After two uncertain years in India, where he was unable to start a successful law practice, Gandhi moved to South Africa in 1893 to represent an Indian me

{'input': 'Tell me a few key things about Mahatma Gandhi.',
 'output': "He was an Indian lawyer, anti-colonial nationalist, and political ethicist, he employed nonviolent resistance to lead the successful campaign for India's independence from British rule, he is considered the Father of the Nation in India, and he was assassinated on January 30, 1948, by a Hindu nationalist."}

{'input': 'Tell me a few key things about Mahatma Gandhi.',
 'output': "He was an Indian lawyer, anti-colonial nationalist, and political ethicist, he employed nonviolent resistance to lead the successful campaign for India's independence from British rule, he is considered the Father of the Nation in India, and he was assassinated on January 30, 1948, by a Hindu nationalist."}

##### Self-ask with Search
This agent utilizes the Intermediate Answer tool for self-asking questions.
Allows LLM to give answers to a question it was not explicitly trained on. The model might not have the direct answer to a question, but answers to sub-questions will exist in the LLM dataset. It achieves this by obtaining Intermediate Answers.

In [24]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
tools =[
    Tool(
        func=search.run,
        name="Intermediate Answer",
        description="A search engine that can be used to find information on the web."
    )
]
agent = initialize_agent(
    tools,
    llm,
    agent="self-ask-with-search",
    verbose=True,
    handle_parsing_errors=True
)

In [25]:
print(agent.invoke("Question: Who was the president of USA when the first Moom landing took place?"))



> Entering new AgentExecutor chain...
Yes.
Follow up: When did the first moon landing take place?
Intermediate answer: The Luna programme was the first successful lunar programme, its Luna 1 (1959) being the first partially successful lunar mission The first image taken of the far side of the Moon, returned by Luna 3 (1959). Missions to the Moon have been numerous and include some of the earliest space missions, conducting exploration of the Moon since 1959.. The first partially successful lunar mission was ... Watch highlights from the Apollo 11 mission including the launch on July 16, 1969, the landing of the lunar module, Neil Armstrong's first steps on the Moon, splashdown, and more. Watch on NASA+ 20 July 1969—Astronaut Edwin E. Aldrin Jr., lunar module pilot, walks on the surface of the moon near the leg of the Lunar Module (LM) "Eagle ... Witness Neil Armstrong take humankind's first steps on the lunar surface during NASA's Apollo 11 mission Perhaps the most famous of all spac